# IMPORTS - INITIALIZATION

In [1]:
import ee, eemont
import xee, wxee
import xarray as xr
import numpy as np

import os
import pandas as pd

import geopandas as gpd
import geemap
import geemap.colormaps as geecm

import matplotlib.pyplot as plt
import cmocean

In [2]:
import ipywidgets

In [6]:
ee.Authenticate()
ee.Initialize()
wxee.Initialize()

# Import of index and Geom data


In [7]:
years = range(1984, 2025)

In [8]:

# Importing Index repository and setting Time start to avoid xarray error
Landsat_annualCollection_index= ee.ImageCollection([
    ee.Image(f"projects/ee-roniritzganem/assets/stage_carrtel_2025/landsat_annual_median_index/Landsat_{year}").select('GBR','TIR','NDWI').set("system:time_start", ee.Date(f"{year}"))
    for year in years
])

In [10]:
fc_lia = ee.FeatureCollection('users/aguerou/ice_and_life/carto_h1b/lia_shp/glaciers_1850_final_mars_2025') # demander les références
geom_lia=fc_lia.geometry()

geom_alpes_FR = ee.FeatureCollection('users/aguerou/ice_and_life/erable_2025/data_ancillary/alpes_francaises')

dem_glo30 = ee.ImageCollection("COPERNICUS/DEM/GLO30")

# Parameters 

In [11]:
scale = 30 #Landsat 30m
projection = "EPSG:4326" #GEE le plus stable avec ce SRC

In [12]:
dem = ee.ImageCollection(dem_glo30).filterBounds(fc_lia).select("DEM")
dem_mosaic = dem.mosaic().clip(fc_lia)
SlopeThresh = 20

In [13]:
# 3 palettes glacier avec nuances différentes (au choix ici pour exemple)
visGLACIER_1 = {"min": 0, "max": 1, "palette": ["#808080", "#228B22"]}   # Vert forêt
visGLACIER_2 = {"min": 0, "max": 1, "palette": ["#808080", "#FF00FF"]}   # Magenta
visGLACIER_3 = {"min": 0, "max": 1, "palette": ["#808080", "#00FFFF"]}   # Cyan
visGLACIER_4 = {"min": 0, "max": 1, "palette": ["#808080", "#FF0000"]}   # Rouge vif

visWATER = {"min": 0, "max": 1, "palette": ["#808080", "#FFA500"]}  # Orange
vis_params_temp = geecm.get_palette('coolwarm', n_class=100)
visTIR = {'min': -5,'max': 10,'palette': vis_params_temp}

# FUNCTIONS

In [14]:
def build_band_image(band_name,collection):
    def select_band(img):
        return img.select([band_name]) \
                  .copyProperties(img, img.propertyNames())
    
    return collection.map(select_band)

In [15]:
def select_band(img):
    years = range(1984, 2025)
    for year in years:
        img.copyProperties(img, img.propertyNames()) \
                    .set("system:time_start", ee.Date(f"{year}"))
    return img
    


In [16]:
def convert_temp_to_deg(image):
    image_degC = image.select('TIR') \
                      .subtract(273.15) \
                      .rename('tempC') \
                      .copyProperties(image, ['system:time_start'])
    return image_degC

In [17]:
def join_ndwi_tir(image_ndwi):
    year = image_ndwi.get('year')
    
    matching_tir = TIR_TS_filled_smooth.filter(ee.Filter.eq('year', year)).first()
    
    combined = image_ndwi.addBands(matching_tir)
    
    return combined

In [18]:
def get_slope(image):
    return ee.Terrain.slope(image)

In [19]:
slope = dem.map(get_slope).mosaic().clip(fc_lia) \
#need to map on single image for slope function to work fine
slope_mask = slope.lt(SlopeThresh)

In [20]:
def classifyGlacier(image, seuil):
    temp = image.select('GBR').gt(seuil)
    mask = temp.where(temp.gte(1), 1)
    return image.addBands(mask.rename('GLACIER').toInt())

def classifyWater(image, seuil):
    temp = image.select('NDWI').gt(seuil)
    temp2 = temp.where(temp.gte(1), 2)
    mask = temp2.updateMask(slope_mask).unmask(0)

    # Temperature mask
    Temp_convert= image.select('TIR').subtract(273.15)
    Temp_mask = Temp_convert.updateMask(Temp_convert.gt(-1).And(Temp_convert.lt(20)))
    mask2 = mask.updateMask(Temp_mask).unmask(0)
    return image.addBands(mask2.rename('WATER').toInt())

-------------------------------------------------------------------------------------

# Processing


In [21]:
 # Creating an IC for each Index/Band containing the 41 studied years
gbr_image = build_band_image('GBR',Landsat_annualCollection_index)
tir_image = build_band_image('TIR', Landsat_annualCollection_index)
ndwi_image = build_band_image('NDWI', Landsat_annualCollection_index)

# # K to C conversion, if not applied during classifiaction
temp_image=tir_image.map(convert_temp_to_deg)

Time series using wxee

In [22]:
GBR_TS=gbr_image.wx.to_time_series()
TIR_TS=temp_image.wx.to_time_series()
NDWI_TS=ndwi_image.wx.to_time_series()

In [24]:
GBR_TS_filled=GBR_TS.rolling_time?

Signature:
GBR_TS.rolling_time(
    window: int,
    unit: str,
    align: str = 'left',
    min_observations: int = 1,
    reducer: Optional[Any] = None,
    keep_bandnames: bool = True,
) -> 'TimeSeries'
Docstring:
Apply a rolling reducer over the time dimension. Rolling windows are calculated around each image,
so if images are irregularly spaced in time, the windows will be as well. As long as the minimum
observations are met in each window, the output time series will contain the same number of images as
the input, with each image reduced over its surrounding window.

Parameters
----------
window : int
    The number of time units to include in each rolling period.
unit : str
    The time frequency of the window. One of "hour", "day", "week", "month", "year".
align : str, default "left"
    The start location of the rolling window, relative to the primary image time. One of "left", "center",
    "right". For example, a 3-day left-aligned window will include all images up to (but n

Gap Filling with mean values (T=3 years)

In [43]:
GBR_TS_filled=GBR_TS.fill_gaps(window=3, unit='year', reducer=ee.Reducer.median())
TIR_TS_filled=TIR_TS.fill_gaps(window=3, unit='year', reducer=ee.Reducer.median())
NDWI_TS_filled=NDWI_TS.fill_gaps(window=3, unit='year', reducer=ee.Reducer.median())

Smoothing with wxee, T=5 years

In [44]:
GBR_TS_filled_smooth=GBR_TS_filled.rolling_time(window=5, unit= 'years', reducer=ee.Reducer.median())
TIR_TS_filled_smooth=TIR_TS_filled.rolling_time(window=5, unit= 'years', reducer=ee.Reducer.median())
NDWI_TS_filled_smooth=NDWI_TS_filled.rolling_time(window=5, unit= 'years', reducer=ee.Reducer.median())

-------------------------------------------------------------------------------------

# Exporting TS data

In [45]:
# Loop through each image in the annualCollection and export it
collection= TIR_TS_filled_smooth
for i in range(collection.size().getInfo()):
    image = ee.Image(collection.toList(collection.size()).get(i))
    
    # Export the image
    geemap.ee_export_image_to_asset(
        image=image,
        description=f'TempC_TS_{1984+i}',
        assetId=f'projects/ee-roniritzganem/assets/stage_carrtel_2025/Time_series_GBR_NDWI_TIR_vdef/TempC/TIR_TS_filled_smooth_{1984+i}',
        region=geom_lia,
        scale=scale,
        maxPixels=1e13,
        crs=projection
        )

In [23]:
# Loop through each image in the annualCollection and export it
collection= GBR_TS_filled_smooth
for i in range(collection.size().getInfo()):
    image = ee.Image(collection.toList(collection.size()).get(i))
    
    
    # Export the image
    geemap.ee_export_image_to_asset(
        image=image,
        description=f'GBR_TS_{1984+i}',
        assetId=f'projects/ee-roniritzganem/assets/stage_carrtel_2025/Time_series_GBR_NDWI_TIR_vdef/GBR/GBR_TS_filled_smooth_{1984+i}',
        region=geom_lia,
        scale=scale,
        maxPixels=1e13,
        crs=projection
    )

In [24]:
# Loop through each image in the annualCollection and export it
collection= NDWI_TS_filled_smooth
for i in range(collection.size().getInfo()):
    image = ee.Image(collection.toList(collection.size()).get(i))
    
    
    # Export the image
    geemap.ee_export_image_to_asset(
        image=image,
        description=f'NDWI_TS_{1984+i}',
        assetId=f'projects/ee-roniritzganem/assets/stage_carrtel_2025/Time_series_GBR_NDWI_TIR_vdef/NDWI/NDWI_TS_filled_smooth_{1984+i}',
        region=geom_lia,
        scale=scale,
        maxPixels=1e13,
        crs=projection
    )

-------------------------------------------------------------------------------------